# Setup

### command line

In [1]:
# import the kaggle.json from kaggle API into colab
# do this command

# install kaggle library
!pip install kaggle
# make a directory named .kaggle
!mkdir ~/.kaggle
# copy the kaggle.json into this new directory
!cp kaggle.json ~/.kaggle/
# alocate the required permission for this file
!chmod 600 ~/.kaggle/kaggle.json
# download dataset
!kaggle datasets download doctrinek/oxford-iiit-cats-extended-10k

 99% 984M/993M [00:10<00:00, 135MB/s]
100% 993M/993M [00:10<00:00, 98.7MB/s]


In [2]:
!pip install split-folders
!pip install pillow

### import libraries

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Input
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import pathlib, zipfile, os, splitfolders, datetime
from PIL import Image

### setup tensorboard

In [18]:
%load_ext tensorboard

In [19]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [20]:
!rm -rf ./logs/

In [21]:
tf.summary.create_file_writer("./logs/")

In [23]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### extract zip file

In [4]:
zip_path = "/content/oxford-iiit-cats-extended-10k.zip"
zip_read = zipfile.ZipFile(zip_path, "r")
zip_read.extractall('/content/dataset')
zip_read.close()

In [5]:
os.listdir('/content/dataset/')

['CatBreedsRefined-v3']

### resize all the images

In [6]:
original_dir = '/content/dataset/CatBreedsRefined-v3'
resized_dir = '/content/dataset/resized'

if os.path.exists(resized_dir):
  import shutil
  shutil.rmtree(resized_dir)


os.makedirs(resized_dir, exist_ok=True)

In [7]:
target_size=(224,224)

for folder in os.listdir(original_dir):
  sub_dir = os.path.join(original_dir, folder)
  resized_img_path = os.path.join(resized_dir, folder)
  os.makedirs(resized_img_path, exist_ok=True)

  for filename in os.listdir(sub_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
      image_path = os.path.join(sub_dir, filename)
      try:
        img = Image.open(image_path)
      except Exception as e:
        print(f"error opening image {filename} : {e}")
        continue

      resized_img = img.resize(target_size, Image.ANTIALIAS)
      save_path = os.path.join(resized_img_path, filename)

      try:
        resized_img.save(save_path, 'jpeg')
      except Exception as e:
        print(f"error saving resized image {filename} : {e}")
        break

<ipython-input-7-3481b1368359>:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize(target_size, Image.ANTIALIAS)


In [8]:
os.listdir(resized_dir)

['Bengal',
 'Siamese',
 'Maine Coon',
 'Ragdoll',
 'Bombay',
 'Russian Blue',
 'Abyssinian',
 'Persian',
 'Birman',
 'Bristish Shorthair',
 'Egyptian Mau',
 'Sphynx']

### split into train and validation

In [9]:
splitfolders.ratio(resized_dir, output='/content/dataset/project', seed=6969, ratio=(0.8, 0.2))
train_dir = '/content/dataset/project/train'
validation_dir = '/content/dataset/project/val'

Copying files: 10257 files [00:02, 4905.09 files/s]


### explore data samples

In [10]:
def total_sample(directory):
  total = 0
  for folder in os.listdir(directory):
    folder_path = os.path.join(directory, folder)
    total += len(os.listdir(folder_path))

  return total

train_sample_length = total_sample(train_dir)
validation_sample_length = total_sample(validation_dir)
print(f"The train directory has {train_sample_length} samples")
print(f"The validation directory has {validation_sample_length} samples")
print(f"Which in total makes it {train_sample_length + validation_sample_length} samples")

The train directory has 8202 samples
The validation directory has 2055 samples
Which in total makes it 10257 samples


# Preprocess Data

In [44]:
batch_size = 64

train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    class_mode='categorical',
    target_size=target_size,
    batch_size=batch_size
)

validation_datagen = ImageDataGenerator(
    rescale=1.0/255
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    class_mode='categorical',
    target_size=target_size,
    batch_size=batch_size
)

Found 8202 images belonging to 12 classes.
Found 2055 images belonging to 12 classes.


# Model Creation

### callback function
- stops when acc & val_acc is >= 92%
- stops when acc / val_acc < max_acc for limit_acc epochs
- stops when loss / val_loss > 0.75 for limit_loss epochs


In [26]:
class SantaiDuluGakSih(tf.keras.callbacks.Callback):
  def __init__(self, sabar_acc=10, sabar_loss=10):
    super(SantaiDuluGakSih, self).__init__()
    self.sabar_acc = sabar_acc
    self.sabar_loss = sabar_loss
    self.limit_acc = sabar_acc
    self.limit_loss = sabar_loss
    self.max_acc = 0
    self.max_val_acc = 0

  def on_epoch_end(self, epoch, logs={}):
    self.max_acc = logs.get('accuracy') if logs.get('accuracy') > self.max_acc else self.max_acc

    self.max_val_acc = logs.get('val_accuracy') if logs.get('val_accuracy') > self.max_val_acc else self.max_val_acc

    if logs.get('accuracy')<self.max_acc or logs.get('val_accuracy')<self.max_val_acc:
      self.sabar_acc -= 1
    else:
      self.sabar_acc += 1

    if logs.get('loss')>0.75 or logs.get('val_loss')>0.75:
      self.sabar_loss -= 1
    else:
      self.sabar_loss += 1

    if self.sabar_acc == 0:
      print(f"The model accuracy has been below {self.max_acc} for {self.limit_acc} epochs, Stopping training immediatly!!!")
      self.model.stop_training = True
    elif self.sabar_loss == 0:
      print(f"The model loss has been above 75% for {self.limit_loss} epochs, Stopping training immediatly!!!")
      self.model.stop_training = True
    elif self.max_acc >= 0.92 and self.max_val_acc >= 0.92:
      print(f"The model accuracy has reached 92%, stopping training")
      self.model.stop_training = True

### transfer learning using MobileNetV2

In [27]:
pre_trained_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_tensor=Input(shape=(224,224,3))
)

for layer in pre_trained_model.layers:
  layer.trainable = False

last_output = pre_trained_model.output

In [45]:
x = Conv2D(32, (3,3), activation="relu")(last_output)
x = MaxPooling2D(2,2)(x)
x = Dropout(0.4)(x)

x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dense(32, activation="relu")(x)
output_layer = Dense(12, activation="softmax")(x)

model = Model(inputs=pre_trained_model.input, outputs=output_layer)

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']      

In [46]:
int_lr = 1e-4
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=int_lr),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [47]:
berhenti_bang = SantaiDuluGakSih(sabar_acc=5, sabar_loss=10)
modelku = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[berhenti_bang, tensorboard_callback],
    verbose=2
)

Epoch 1/30
129/129 - 109s - loss: 2.1276 - accuracy: 0.2732 - val_loss: 1.4274 - val_accuracy: 0.5207 - 109s/epoch - 844ms/step
Epoch 2/30
129/129 - 103s - loss: 1.4088 - accuracy: 0.5156 - val_loss: 0.9760 - val_accuracy: 0.6881 - 103s/epoch - 802ms/step
Epoch 3/30
129/129 - 104s - loss: 1.0680 - accuracy: 0.6378 - val_loss: 0.7544 - val_accuracy: 0.7513 - 104s/epoch - 810ms/step
Epoch 4/30
129/129 - 102s - loss: 0.9133 - accuracy: 0.6909 - val_loss: 0.6518 - val_accuracy: 0.7825 - 102s/epoch - 793ms/step
Epoch 5/30


KeyboardInterrupt: ignored

# Model evaluation

### plot loss and accuracy

In [ ]:
# plot loss
plt.plot(modelku.history['loss'])
plt.plot(modelku.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# plot acc
plt.plot(modelku.history['accuracy'])
plt.plot(modelku.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

### show tensorboard

In [ ]:
%tensorboard --logdir logs/fit

# Save model